# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [16]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-28","2019-03-03",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-28 to 2019-03-03
536


In [17]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [18]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='wnLemm')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='wnLemm')

content
wnLemm
Binary Encoding
True
title
wnLemm
Binary Encoding
True


In [19]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Tesla's store-shuttering strategy may pull the...   
1  L Brands plans to close roughly 53 Victoria's ...   
2  US stocks rise on hopes for a trade deal with ...   
3  Stocks making the biggest moves midday: Gap, T...   
4    Week ahead: US jobs, retail earnings, ECB rates   

                                         description  
0  Tesla Inc's sudden decision to shutter the bul...  
1  L Brands, owner of Victoria's Secret and Bath ...  
2  NEW YORK (AP) — U.S. stocks are rising Friday,...  
3  These are the stocks posting the largest moves...  
4  While US-China trade talks continue to hold ce...  


In [20]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 321/321 [01:55<00:00,  1.72it/s]


In [21]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 321/321 [00:01<00:00, 244.82it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 321/321 [00:01<00:00, 252.46it/s]


          san  francisco  tesla    sudden  decision  shutter      bulk  \
0    0.041667   0.071429   1.05  0.166667  0.021739      0.1  0.166667   
1    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
2    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
3    0.000000   0.000000   0.10  0.000000  0.000000      0.0  0.000000   
4    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
5    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
6    0.000000   0.000000   0.70  0.000000  0.000000      0.0  0.000000   
7    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
8    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
9    0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
10   0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
11   0.000000   0.000000   0.00  0.000000  0.000000      0.0  0.000000   
12   0.000000   0.000000   0.05  0.000

[321 rows x 16778 columns]


In [22]:
articleDB.head()

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,context,tags,tags_top_5,related_articles
0,0.415244,0.584756,https://in.reuters.com/article/tesla-solar-idI...,1,Tesla's store-shuttering strategy may pull the...,Tesla Inc's sudden decision to shutter the bul...,reuters,2019-03-02T01:18:25Z,san francisco tesla sudden decision shutter bu...,SAN FRANCISCO (Reuters) - Tesla Inc’s sudden d...,SAN FRANCISCO (Reuters) - Tesla Inc s sudden d...,"SAN FRANCISCO, Tesla sudden decision, raises, ...","san francisco, tesla sudden decision, red flag...","principal analyst,wide layoffs,residential sol...","101, 34, 27"
1,0.423063,0.576937,https://www.cnbc.com/2019/02/28/l-brands-plans...,1,L Brands plans to close roughly 53 Victoria's ...,"L Brands, owner of Victoria's Secret and Bath ...",cnbc,2019-02-28T12:45:00Z,l brands plans close roughly victoria secret s...,L Brands plans to close roughly 53 Victoria's ...,L Brands plans to close roughly 53 Victoria's ...,"Brands, plans, Victoria, lingerie retailer, ne...","victoria, lingerie retailer, newer rivals, bat...","cfo stuart burgdoerfer,third love,latest perio...","35, 15, 24"
2,0.447545,0.552455,https://apnews.com/644841507f3b4bae81f70cc3169...,1,US stocks rise on hopes for a trade deal with ...,"NEW YORK (AP) — U.S. stocks are rising Friday,...",associated-press,2019-03-01T15:53:09Z,new york s stocks rising reversing three day l...,"NEW YORK (AP) U.S. stocks are rising Friday, r...","NEW YORK (AP) U.S. stocks are rising Friday, r...","YORK, U.S. stocks, rising, day losing streak, ...","york, u.s. stocks, day losing streak, investor...","chinese products,separate company,footwear,str...","43, 44, 94"
3,0.449331,0.550669,https://www.cnbc.com/2019/03/01/stocks-making-...,1,"Stocks making the biggest moves midday: Gap, T...",These are the stocks posting the largest moves...,cnbc,2019-03-01T17:14:00Z,tesla shares tesla fell percent company underw...,Tesla Shares of Tesla fell 7.58 percent after ...,Tesla Shares of Tesla fell 7.58 percent after ...,"Tesla Shares, Tesla, fell, company, standard M...","tesla shares, tesla, standard model, barclays ...","electric car maker,advancing many new initiati...","49, 230, 50"
4,0.450367,0.549633,https://www.ft.com/content/66191714-3c47-11e9-...,1,"Week ahead: US jobs, retail earnings, ECB rates",While US-China trade talks continue to hold ce...,financial-times,2019-03-01T22:31:37Z,china trade talks continue hold centre stage ...,While US-China trade talks continue to hold ce...,While US-China trade talks continue to hold ce...,"China trade talks, centre stage, investors, fo...","china trade talks, centre stage, investors, ec...","earnings reports,particular interest,monetary ...","11, 31, 40"


In [23]:
import frontPage as fp
frontpage, articleDBB = fp.FrontPage(articleDB, trendingTermsDB)

                                               title            source  \
0  Tesla's store-shuttering strategy may pull the...           reuters   
1  L Brands plans to close roughly 53 Victoria's ...              cnbc   
2  US stocks rise on hopes for a trade deal with ...  associated-press   
3  Stocks making the biggest moves midday: Gap, T...              cnbc   
4    Week ahead: US jobs, retail earnings, ECB rates   financial-times   
5                       Saks Takes Back Fifth Avenue           fortune   
6  A month ago, Elon Musk thought Tesla would be ...              cnbc   
7  J.C. Penney Once Again Learns It Has to Listen...           fortune   
8  America’s Trucker Shortage Is About to Hit Con...           fortune   
9  Walmart and Affirm are partnering to offer poi...  business-insider   

                   date                                        origContent  
0  2019-03-02T01:18:25Z  SAN FRANCISCO (Reuters) - Tesla Inc’s sudden d...  
1  2019-02-28T12:45:00Z  L Bran

In [24]:
frontpage, articleDBBB = FrontPagee(articleDB, trendingTermsDB)

                                               title            source  \
0  Tesla's store-shuttering strategy may pull the...           reuters   
1  L Brands plans to close roughly 53 Victoria's ...              cnbc   
2  US stocks rise on hopes for a trade deal with ...  associated-press   
3  Stocks making the biggest moves midday: Gap, T...              cnbc   
4    Week ahead: US jobs, retail earnings, ECB rates   financial-times   
5                       Saks Takes Back Fifth Avenue           fortune   
6  A month ago, Elon Musk thought Tesla would be ...              cnbc   
7  J.C. Penney Once Again Learns It Has to Listen...           fortune   
8  America’s Trucker Shortage Is About to Hit Con...           fortune   
9  Walmart and Affirm are partnering to offer poi...  business-insider   

                   date                                        origContent  
0  2019-03-02T01:18:25Z  SAN FRANCISCO (Reuters) - Tesla Inc’s sudden d...  
1  2019-02-28T12:45:00Z  L Bran

NameError: name 'json' is not defined

In [ ]:
def FrontPagee(articleDB, trendingTermsDB):
    # number of top articles
    # todo; change to only 'predicted relevant' articles
    numArts = 10
    
    # get articles
    art = articleDB.iloc[0:numArts][['title','source', 'date', 'origContent']]
    print(art)
    art['tags'] = list(map(lambda x: x.split(','), articleDB.iloc[0:numArts]['tags_top_5']))
    # grab related article IDs
    rel_arts = list(map(lambda x: x.split(','), articleDB.iloc[0:numArts]['related_articles']))
    print(rel_arts)
    # use IDs to grab related article title, source, url, turn into little dictionaries and add to art
    art['related_articles'] = list(map(lambda num: articleDB.iloc[num][['title','source','url']].to_dict(orient='records'), rel_arts))
    
    artDict = art.to_dict(orient='records')
        
    # get top terms
    tuples = [tuple(x) for x in trendingTermsDB.values]
    topTerms = tuples[:10]
    
    # output final json
    frontpage = {"topterms":topTerms, "articles":artDict}
    print(type(frontpage))
    with open("frontPagee.json", "w") as write_file:
        json.dump(frontpage, write_file)
    
    return frontpage

In [ ]:
articleDB